In [1]:
#Variabilil globali
width = 256
height = 256
mean_pre_trained =[0.485, 0.456, 0.406]
std_pre_trained =[0.229, 0.224, 0.225]

# TODO:
- Far partire Visdom
- Data agumentation
- Aumentare il knn con data agumentation.
- Allenare con nostre foto.



# Domande al prof:
- Perchè cambiando le dimensioni, aumentandole e diminuendole, l'accuracy cambia anche di molto?

# Import necessari

In [2]:
import torchvision.models as models
from torch.utils.data.dataset import Dataset
from PIL import Image
from os import path
from torchvision import transforms
import numpy as np
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier as KNN
from torch.autograd import Variable
from sklearn.metrics import accuracy_score
from torch.optim import SGD
from torch.autograd import Variable
from torchnet.logger import VisdomPlotLogger, VisdomSaver
from torchnet.meter import AverageValueMeter

# Classe per la gestione del dataset

In [3]:
np.random.seed(1234)
torch.random.manual_seed(1234);

class ScenesDataset(Dataset):
    def __init__(self,base_path,txt_list,transform=None):
        #conserviamo il path alla cartella contenente le immagini
        self.base_path=base_path
        #carichiamo la lista dei file
        #sarà una matrice con n righe (numero di immagini) e 2 colonne (path, etichetta)
        self.images = np.loadtxt(txt_list,dtype=str,delimiter=',')
        #print("self.images ha i seguenti elementi:", len(self.images))
        #conserviamo il riferimento alla trasformazione da applicare
        self.transform = transform
    def __getitem__(self, index):
        #print("Get item numero -->", index)
        #recuperiamo il path dell'immagine di indice index e la relativa etichetta
        f,c = self.images[index]
        #carichiamo l'immagine utilizzando PIL e facciamo il resize a 3 canali.
        im = Image.open(path.join(self.base_path, f)).convert("RGB")
        
        #Resize:
        im = im.resize((width,height))
        #se la trasfromazione è definita, applichiamola all'immagine
        if self.transform is not None:
            im = self.transform(im)       
        
        #convertiamo l'etichetta in un intero
        label = int(c)
        #restituiamo un dizionario contenente immagine etichetta
        #print("Mentre creo il tutto, label vale-->", label, ", name vale -->", f)
        return {'image' : im, 'label':label, 'name': f}
    #restituisce il numero di campioni: la lunghezza della lista "images"
    def __len__(self):
        #print("Ho invocato len, vale-->", len(self.images))
        return len(self.images)

# Funzione che ci restituisce media e varianza di un dataset.
- Prende in input il dataset.

In [4]:
def get_mean_devst(dataset):
    m = np.zeros(3)
    for sample in dataset:
        m+= np.array(sample['image'].sum(1).sum(1)) #accumuliamo la somma dei pixel canale per canale
    #dividiamo per il numero di immagini moltiplicato per il numero di pixel
    m=m/(len(dataset)*width*height)
    #procedura simile per calcolare la deviazione standard
    s = np.zeros(3)
    for sample in dataset:
        s+= np.array(((sample['image']-torch.Tensor(m).view(3,1,1))**2).sum(1).sum(1))
    s=np.sqrt(s/(len(dataset)*width*height))
    print("Medie",m)
    print("Dev.Std.",s)
    return m, s

# Prendiamo media e varianza per normalizzare successivamente i dati

In [5]:
#dataset = ScenesDataset('Dataset_base','train.txt',transform=transforms.ToTensor())
#m, s = get_mean_devst(dataset)

# Funzione per calcolare l'accuracy.

In [6]:
# Prende in input l'array di feature e il classificatore(knn.) 
def accuracy(classifier, samples):
    right_pred = 0
    for i in range(len(samples)):
        pred_label = classifier.predict(samples[i]["feature"].cpu().detach().numpy().reshape(1, -1))
        if pred_label[0] == samples[i]["label"]:
            right_pred += 1
            
    return float(right_pred)/len(samples)

# Funzione per estrazione feature:
- In input dataset (dataloader) e rete.

In [7]:
def extract_features(dataset, net):
    #Presa ogni riga del dataloader li passa alla net senza attivare il layer di classificazione
    feature_dataset = []
    print("Avviato extract_feature.")
    for i, dataset_train in enumerate(dataset):
        x=Variable(dataset_train['image'], requires_grad=False)
        y=Variable(dataset_train['label'])
        x, y = x.cpu(), y.cpu()
        if torch.cuda.is_available():
            x, y = x.cuda(), y.cuda()
            #print("Con cuda")
        output = net(x)
        feature_dataset.append({"label": dataset_train['label'], "feature":output, "name": dataset_train['name']})
    return feature_dataset

# Funzione per creare l'input per l'oggetto dataframe:
- In input dataset (dataloader) e rete.

In [8]:
def get_dataframe(dataset, net):
    print("Avviato get_dataframe.")
    feature_dataset = extract_features(dataset, net)  
    feature_dataset_matrix = np.zeros((len(feature_dataset), len(feature_dataset[0]["feature"][0])))    
    #Qui abbiamo nelle righe tutte le immagini, nella lable feature tutte le 9000 colonne, ossia le feature.
    label_array = np.zeros(len(feature_dataset))
    for i in range(0, len(feature_dataset)):#302
        for j in range(0, len(feature_dataset[0]["feature"][0])):#9206
            if j == 0:#salviamo la y finale nell'array label_array
                label_array[i] = feature_dataset[i]['label'][0]
                #print(i, end= " ")
            feature_dataset_matrix[i][j] =feature_dataset[i]["feature"][0][j] 

    return feature_dataset_matrix, label_array

In [9]:

def train_classification(model, train_loader, test_loader, lr=0.001, epochs=20, momentum=0.8):
    criterion = nn.CrossEntropyLoss()
    optimizer = SGD(model.parameters(),lr, momentum=momentum)
    loaders = {'train':train_loader, 'test':test_loader}
    losses = {'train':[], 'test':[]}
    accuracies = {'train':[], 'test':[]}
    
    
    exp_name='experiment_50_epochs'
    loss_meter = AverageValueMeter()
    acc_meter = AverageValueMeter()
    loss_logger = VisdomPlotLogger('line', env=exp_name, opts={'title': 'Loss', 'legend':['train','test']})
    acc_logger = VisdomPlotLogger('line', env=exp_name, opts={'title': 'Accuracy','legend':['train','test']})
    visdom_saver = VisdomSaver(envs=[exp_name])

    if torch.cuda.is_available():
        model=model.cuda()
    for e in range(epochs):
        #print("Primo ciclo for.")
        for mode in ['train', 'test']:
            #print("Secondo ciclo for.")
            
            loss_meter.reset()
            acc_meter.reset()
            
            if mode=='train':
                model.train()
            else:
                model.eval()
            epoch_loss = 0
            epoch_acc = 0
            samples = 0
            #print("Mode-->",mode)
            #print("Enumerate-->", loaders[mode])
            for i, batch in enumerate(loaders[mode]):
                #trasformiamo i tensori in variabili
                x=Variable(batch['image'], requires_grad=(mode=='train'))
                y=Variable(batch['label'])
                if torch.cuda.is_available():
                    x, y = x.cuda(), y.cuda()
                    print("Con cuda")
                #else:
                    #print("Senza cuda")
                output = model(x)
                #print(type(output))
                #print(output)
                l = criterion(output,y)
                if mode=='train':
                    l.backward()
                    optimizer.step()
                    optimizer.zero_grad()
                #print("L-->",l.item())
                acc = accuracy_score(y.cpu().data,output.cpu().max(1)[1].data)
                epoch_loss+=l.data.item()*x.shape[0]
                epoch_acc+=acc*x.shape[0]
                samples+=x.shape[0]
                print ("\r[%s] Epoch %d/%d. Iteration %d/%d. Loss: %0.2f. Accuracy: %0.2f\t\t\t\t\t" % \
                (mode, e+1, epochs, i, len(loaders[mode]), epoch_loss/samples, epoch_acc/samples),
                epoch_loss/samples,
                epoch_acc/samples,
                losses[mode].append(epoch_loss))
                accuracies[mode].append(epoch_acc)
                n = batch['image'].shape[0]
                loss_meter.add(l.item()*n,n)
                acc_meter.add(acc*n,n)
                loss_logger.log(e+(i+1)/len(loaders[mode]), loss_meter.value()[0], name=mode)
                acc_logger.log(e+(i+1)/len(loaders[mode]), acc_meter.value()[0], name=mode)


            loss_logger.log(e+1, loss_meter.value()[0], name=mode)
            acc_logger.log(e+1, acc_meter.value()[0], name=mode)
            
            #print("Fine secondo ciclo for")
        print("\r[%s] Epoch %d/%d. Iteration %d/%d. Loss: %0.2f. Accuracy: %0.2f\t\t\t\t\t" % \
        (mode, e+1, epochs, i, len(loaders[mode]), epoch_loss, epoch_acc))

    print("Ho finito.")
    #restituiamo il modello e i vari log
    return model, (losses, accuracies)

In [10]:
import numpy as np
def test_model_classification(model, test_loader):
    softmax = nn.Softmax(dim=1)
    model.eval()
    model.cpu()
    preds = []
    gts = []
    for batch in test_loader:
        x=Variable(batch["image"])
        x = x.cpu()
        #applichiamo la funzione softmax per avere delle probabilità
        if torch.cuda.is_available():
            x = x.cuda()
            model.cuda()
        pred = softmax(model(x)).data.cpu().numpy().copy()
        gt = batch["label"].cpu().numpy().copy()
        #print("Pred-->", pred, ", gt-->", gt)
        preds.append(pred)
        gts.append(gt)
        #print(len(preds), len(gts))
    return np.concatenate(preds),np.concatenate(gts)

# Creiamo Oggetto Dataset e dataloader, sia base che data augmentation.

In [11]:
transformss = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean_pre_trained,std_pre_trained)])
barilla_train = ScenesDataset('Dataset_base','train.txt',transform=transformss)
barilla_test = ScenesDataset('Dataset_base','test.txt',transform=transformss)
barilla_train_loader = torch.utils.data.DataLoader(barilla_train, batch_size=10, num_workers=0, shuffle=True)
barilla_test_loader = torch.utils.data.DataLoader(barilla_test, batch_size=10, num_workers=0)

In [12]:
transformss = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean_pre_trained,std_pre_trained)])
barilla_train_da = ScenesDataset('Dataset_data_augmentation','train_data_augmentation.txt',transform=transformss)
barilla_test_da = ScenesDataset('Dataset_data_augmentation','test_data_augmentation.txt',transform=transformss)
barilla_train_loader_da = torch.utils.data.DataLoader(barilla_train_da, batch_size=10, num_workers=0, shuffle=True)
barilla_test_loader_da = torch.utils.data.DataLoader(barilla_test_da, batch_size=10, num_workers=0)

# Utilizziamo AlexNet con i parametri pre-allenati e un numero di classi finali pari a 153

In [13]:
net = models.alexnet(pretrained=True)
net.classifier[6] = nn.Linear(4096, 153) #Numero esatto di classi nel nostro dataset.
sum([p.numel() for p in net.parameters()])

57630681

# Accuracy senza usare il knn

In [14]:
lenet_mnist_predictions, lenet_mnist_gt = test_model_classification(net, barilla_train_loader)
print ("Accuracy aLexNet di train su barillatestloader: %0.2f" % \
accuracy_score(lenet_mnist_gt,lenet_mnist_predictions.argmax(1)))

Accuracy aLexNet di train su barillatestloader: 0.00


In [15]:
lenet_mnist_predictions, lenet_mnist_gt = test_model_classification(net, barilla_test_loader)
print ("Accuracy aLexNet di test su barillatestloader: %0.2f" % \
accuracy_score(lenet_mnist_gt,lenet_mnist_predictions.argmax(1)))

Accuracy aLexNet di test su barillatestloader: 0.01


# Creiamo il dataframe e il knn

In [18]:
knn_1 = KNN(n_neighbors=1)
knn_3 = KNN(n_neighbors=3)
knn_5 = KNN(n_neighbors=5)

if torch.cuda.is_available():
    net = net.cuda()
    torch.cuda.empty_cache()
net.eval()

barilla_train_loader_OB = torch.utils.data.DataLoader(barilla_train, batch_size=1, num_workers=0, shuffle=True)
barilla_test_loader_OB = torch.utils.data.DataLoader(barilla_test, batch_size=1, num_workers=0)


input_for_datafram_train, label_array_train = get_dataframe(barilla_train_loader_OB, net)
df = pd.DataFrame(input_for_datafram_train)
knn_1.fit(df, label_array_train)
knn_3.fit(df, label_array_train)
knn_5.fit(df, label_array_train)

Avviato get_dataframe.
Avviato extract_feature.


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [20]:
feature_test = extract_features(barilla_test_loader_OB, net)
print("Accuracy con rete preallenata e dataset base.")
print("1nn-->",accuracy(knn_1, feature_test))
print("3nn-->",accuracy(knn_3, feature_test))
print("5nn-->",accuracy(knn_5, feature_test))

Avviato extract_feature.
Accuracy con rete preallenata e dataset base.
1nn--> 0.45394736842105265
3nn--> 0.4407894736842105
5nn--> 0.40131578947368424


# -----------------------------------------------------------------------------------------------

# Utilizziamo AlexNet con i parametri pre-allenati e un numero di classi finali pari a 1000 (valore di default)


In [21]:
net = models.alexnet(pretrained=True)
sum([p.numel() for p in net.parameters()])

61100840

# Accuracy senza usare il knn

In [22]:
lenet_mnist_predictions, lenet_mnist_gt = test_model_classification(net, barilla_train_loader)
print ("Accuracy aLexNet di train su barillatestloader: %0.2f" % \
accuracy_score(lenet_mnist_gt,lenet_mnist_predictions.argmax(1)))

Accuracy aLexNet di train su barillatestloader: 0.00


In [23]:
lenet_mnist_predictions, lenet_mnist_gt = test_model_classification(net, barilla_test_loader)
print ("Accuracy aLexNet di test su barillatestloader: %0.2f" % \
accuracy_score(lenet_mnist_gt,lenet_mnist_predictions.argmax(1)))

Accuracy aLexNet di test su barillatestloader: 0.00


# Creiamo il dataframe e il knn

In [24]:
knn_1 = KNN(n_neighbors=1)
knn_3 = KNN(n_neighbors=3)
knn_5 = KNN(n_neighbors=5)

if torch.cuda.is_available():
    net = net.cuda()
    torch.cuda.empty_cache()
net.eval()

barilla_train_loader_OB = torch.utils.data.DataLoader(barilla_train, batch_size=1, num_workers=0, shuffle=True)
barilla_test_loader_OB = torch.utils.data.DataLoader(barilla_test, batch_size=1, num_workers=0)


input_for_datafram_train, label_array_train = get_dataframe(barilla_train_loader_OB, net)
df = pd.DataFrame(input_for_datafram_train)
knn_1.fit(df, label_array_train)
knn_3.fit(df, label_array_train)
knn_5.fit(df, label_array_train)

Avviato get_dataframe.
Avviato extract_feature.


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [25]:
feature_test = extract_features(barilla_test_loader_OB, net)
print("Accuracy con rete preallenata e dataset base.")
print("1nn-->",accuracy(knn_1, feature_test))
print("3nn-->",accuracy(knn_3, feature_test))
print("5nn-->",accuracy(knn_5, feature_test))

Avviato extract_feature.
Accuracy con rete preallenata e dataset base.
1nn--> 0.4276315789473684
3nn--> 0.4342105263157895
5nn--> 0.4276315789473684


# -----------------------------------------------------------------------------------------------

# Utilizziamo AlexNet riallenando solamente l'ultimo layer con un numero di classi pari a 153 e 20 epoche di allenamento.


In [37]:
net = models.alexnet(pretrained=True)
for param in net.parameters():
    param.requires_grad = False

net.classifier[6] = nn.Linear(4096, 153) #Numero esatto di classi nel nostro dataset.
sum([p.numel() for p in net.parameters()])

57630681

In [39]:
lenet_mnist, lenet_mnist_logs = train_classification(net, epochs=5, train_loader = barilla_train_loader,
                                                     test_loader = barilla_test_loader)

Con cuda
[train] Epoch 1/5. Iteration 0/31. Loss: 1.29. Accuracy: 0.60					 1.285439133644104 0.6 None
Con cuda
[train] Epoch 1/5. Iteration 1/31. Loss: 1.92. Accuracy: 0.40					 1.9203843474388123 0.4 None
Con cuda
[train] Epoch 1/5. Iteration 2/31. Loss: 2.10. Accuracy: 0.37					 2.099344770113627 0.36666666666666664 None
Con cuda
[train] Epoch 1/5. Iteration 3/31. Loss: 2.17. Accuracy: 0.38					 2.1727690398693085 0.375 None
Con cuda
[train] Epoch 1/5. Iteration 4/31. Loss: 2.13. Accuracy: 0.40					 2.1342270374298096 0.4 None
Con cuda
[train] Epoch 1/5. Iteration 5/31. Loss: 2.04. Accuracy: 0.45					 2.0396510561307273 0.45 None
Con cuda
[train] Epoch 1/5. Iteration 6/31. Loss: 2.06. Accuracy: 0.44					 2.0576272181102206 0.44285714285714284 None
Con cuda
[train] Epoch 1/5. Iteration 7/31. Loss: 2.13. Accuracy: 0.44					 2.128670886158943 0.4375 None
Con cuda
[train] Epoch 1/5. Iteration 8/31. Loss: 2.13. Accuracy: 0.44					 2.13158733314938 0.4444444444444444 None
Con cuda
[train]

[train] Epoch 2/5. Iteration 24/31. Loss: 2.04. Accuracy: 0.46					 2.0428746485710145 0.456 None
Con cuda
[train] Epoch 2/5. Iteration 25/31. Loss: 2.04. Accuracy: 0.45					 2.044251513022643 0.45384615384615384 None
Con cuda
[train] Epoch 2/5. Iteration 26/31. Loss: 2.05. Accuracy: 0.46					 2.0466438553951405 0.45555555555555555 None
Con cuda
[train] Epoch 2/5. Iteration 27/31. Loss: 2.05. Accuracy: 0.45					 2.051168286374637 0.45357142857142857 None
Con cuda
[train] Epoch 2/5. Iteration 28/31. Loss: 2.02. Accuracy: 0.46					 2.016170253013742 0.4586206896551724 None
Con cuda
[train] Epoch 2/5. Iteration 29/31. Loss: 2.00. Accuracy: 0.46					 1.995576955874761 0.4633333333333333 None
Con cuda
[train] Epoch 2/5. Iteration 30/31. Loss: 1.99. Accuracy: 0.46					 1.9914108321366721 0.46357615894039733 None
Con cuda
[test] Epoch 2/5. Iteration 0/16. Loss: 0.79. Accuracy: 0.60					 0.785247266292572 0.6 None
Con cuda
[test] Epoch 2/5. Iteration 1/16. Loss: 0.92. Accuracy: 0.70					 0.923

[train] Epoch 4/5. Iteration 1/31. Loss: 1.59. Accuracy: 0.60					 1.588592529296875 0.6 None
Con cuda
[train] Epoch 4/5. Iteration 2/31. Loss: 1.61. Accuracy: 0.63					 1.6110856533050537 0.6333333333333333 None
Con cuda
[train] Epoch 4/5. Iteration 3/31. Loss: 1.66. Accuracy: 0.57					 1.6613702476024628 0.575 None
Con cuda
[train] Epoch 4/5. Iteration 4/31. Loss: 1.54. Accuracy: 0.62					 1.5437485694885253 0.62 None
Con cuda
[train] Epoch 4/5. Iteration 5/31. Loss: 1.52. Accuracy: 0.62					 1.5210461417833965 0.6166666666666667 None
Con cuda
[train] Epoch 4/5. Iteration 6/31. Loss: 1.46. Accuracy: 0.63					 1.4635710716247559 0.6285714285714286 None
Con cuda
[train] Epoch 4/5. Iteration 7/31. Loss: 1.43. Accuracy: 0.64					 1.434534564614296 0.6375 None
Con cuda
[train] Epoch 4/5. Iteration 8/31. Loss: 1.38. Accuracy: 0.67					 1.3825402922100491 0.6666666666666666 None
Con cuda
[train] Epoch 4/5. Iteration 9/31. Loss: 1.42. Accuracy: 0.67					 1.4224936246871949 0.67 None
Con cuda

[train] Epoch 5/5. Iteration 25/31. Loss: 1.45. Accuracy: 0.58					 1.452446855031527 0.5769230769230769 None
Con cuda
[train] Epoch 5/5. Iteration 26/31. Loss: 1.42. Accuracy: 0.59					 1.4249542024400499 0.5851851851851851 None
Con cuda
[train] Epoch 5/5. Iteration 27/31. Loss: 1.43. Accuracy: 0.59					 1.4250179103442602 0.5857142857142857 None
Con cuda
[train] Epoch 5/5. Iteration 28/31. Loss: 1.41. Accuracy: 0.59					 1.4143847555949771 0.5896551724137931 None
Con cuda
[train] Epoch 5/5. Iteration 29/31. Loss: 1.43. Accuracy: 0.58					 1.4338600595792135 0.5766666666666667 None
Con cuda
[train] Epoch 5/5. Iteration 30/31. Loss: 1.43. Accuracy: 0.58					 1.4315038729977134 0.5761589403973509 None
Con cuda
[test] Epoch 5/5. Iteration 0/16. Loss: 0.63. Accuracy: 0.80					 0.6313491463661194 0.8 None
Con cuda
[test] Epoch 5/5. Iteration 1/16. Loss: 0.65. Accuracy: 0.80					 0.6483489871025085 0.8 None
Con cuda
[test] Epoch 5/5. Iteration 2/16. Loss: 0.92. Accuracy: 0.73					 0.9179995

# Accuracy senza usare il knn

In [40]:
lenet_mnist_predictions, lenet_mnist_gt = test_model_classification(net, barilla_train_loader)
print ("Accuracy aLexNet di train su barillatestloader: %0.2f" % \
accuracy_score(lenet_mnist_gt,lenet_mnist_predictions.argmax(1)))

Accuracy aLexNet di train su barillatestloader: 0.83


In [41]:
lenet_mnist_predictions, lenet_mnist_gt = test_model_classification(net, barilla_test_loader)
print ("Accuracy aLexNet di test su barillatestloader: %0.2f" % \
accuracy_score(lenet_mnist_gt,lenet_mnist_predictions.argmax(1)))

Accuracy aLexNet di test su barillatestloader: 0.48


# Creiamo il dataframe e il knn

In [42]:
knn_1 = KNN(n_neighbors=1)
knn_3 = KNN(n_neighbors=3)
knn_5 = KNN(n_neighbors=5)

if torch.cuda.is_available():
    net = net.cuda()
    torch.cuda.empty_cache()
net.eval()

barilla_train_loader_OB = torch.utils.data.DataLoader(barilla_train, batch_size=1, num_workers=0, shuffle=True)
barilla_test_loader_OB = torch.utils.data.DataLoader(barilla_test, batch_size=1, num_workers=0)


input_for_datafram_train, label_array_train = get_dataframe(barilla_train_loader_OB, net)
df = pd.DataFrame(input_for_datafram_train)
knn_1.fit(df, label_array_train)
knn_3.fit(df, label_array_train)
knn_5.fit(df, label_array_train)

Avviato get_dataframe.
Avviato extract_feature.


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [43]:
feature_test = extract_features(barilla_test_loader_OB, net)
print("Accuracy con rete preallenata e dataset base.")
print("1nn-->",accuracy(knn_1, feature_test))
print("3nn-->",accuracy(knn_3, feature_test))
print("5nn-->",accuracy(knn_5, feature_test))

Avviato extract_feature.
Accuracy con rete preallenata e dataset base.
1nn--> 0.4934210526315789
3nn--> 0.4605263157894737
5nn--> 0.40789473684210525


# -----------------------------------------------------------------------------------------------

# Utilizziamo AlexNet riallenando gli ultimi due layer con un numero di classi pari a 153 e 20 epoche di allenamento.


In [26]:
net = models.alexnet(pretrained=True)
for param in net.parameters():
    param.requires_grad = False
    
net.classifier[4] = nn.Linear(4096, 4096) #Numero esatto di classi nel nostro dataset.
net.classifier[6] = nn.Linear(4096, 153) #Numero esatto di classi nel nostro dataset.
sum([p.numel() for p in net.parameters()])

57630681

In [32]:
lenet_mnist, lenet_mnist_logs = train_classification(net, epochs=10, train_loader = barilla_train_loader,
                                                     test_loader = barilla_test_loader)

Con cuda
[train] Epoch 1/20. Iteration 0/31. Loss: 1.79. Accuracy: 0.40					 1.7875999212265015 0.4 None
Con cuda
[train] Epoch 1/20. Iteration 1/31. Loss: 1.19. Accuracy: 0.65					 1.1923076212406158 0.65 None
Con cuda
[train] Epoch 1/20. Iteration 2/31. Loss: 1.17. Accuracy: 0.67					 1.1734988888104756 0.6666666666666666 None
Con cuda
[train] Epoch 1/20. Iteration 3/31. Loss: 1.34. Accuracy: 0.55					 1.3388090878725052 0.55 None
Con cuda
[train] Epoch 1/20. Iteration 4/31. Loss: 1.35. Accuracy: 0.56					 1.3484121203422545 0.56 None
Con cuda
[train] Epoch 1/20. Iteration 5/31. Loss: 1.41. Accuracy: 0.55					 1.4140115280946095 0.55 None
Con cuda
[train] Epoch 1/20. Iteration 6/31. Loss: 1.53. Accuracy: 0.53					 1.5344894698687963 0.5285714285714286 None
Con cuda
[train] Epoch 1/20. Iteration 7/31. Loss: 1.46. Accuracy: 0.55					 1.4630724787712097 0.55 None
Con cuda
[train] Epoch 1/20. Iteration 8/31. Loss: 1.44. Accuracy: 0.57					 1.437069124645657 0.5666666666666667 None
Con cu

[train] Epoch 4/20. Iteration 0/31. Loss: 1.44. Accuracy: 0.60					 1.4438621997833252 0.6 None
Con cuda
[train] Epoch 4/20. Iteration 1/31. Loss: 1.43. Accuracy: 0.60					 1.4319499135017395 0.6 None
Con cuda
[train] Epoch 4/20. Iteration 2/31. Loss: 1.28. Accuracy: 0.70					 1.279105007648468 0.7 None
Con cuda
[train] Epoch 4/20. Iteration 3/31. Loss: 1.08. Accuracy: 0.75					 1.0818181335926056 0.75 None
Con cuda
[train] Epoch 4/20. Iteration 4/31. Loss: 1.07. Accuracy: 0.72					 1.0732860088348388 0.72 None
Con cuda
[train] Epoch 4/20. Iteration 5/31. Loss: 1.05. Accuracy: 0.72					 1.0478570262591045 0.7166666666666667 None
Con cuda
[train] Epoch 4/20. Iteration 6/31. Loss: 1.07. Accuracy: 0.70					 1.0701966456004552 0.7 None
Con cuda
[train] Epoch 4/20. Iteration 7/31. Loss: 1.13. Accuracy: 0.69					 1.1317811757326126 0.6875 None
Con cuda
[train] Epoch 4/20. Iteration 8/31. Loss: 1.21. Accuracy: 0.67					 1.2053610748714871 0.6666666666666666 None
Con cuda
[train] Epoch 4/20. I

[train] Epoch 7/20. Iteration 0/31. Loss: 0.82. Accuracy: 0.60					 0.820212185382843 0.6 None
Con cuda
[train] Epoch 7/20. Iteration 1/31. Loss: 0.60. Accuracy: 0.80					 0.5997117161750793 0.8 None
Con cuda
[train] Epoch 7/20. Iteration 2/31. Loss: 0.49. Accuracy: 0.83					 0.49444634715716046 0.8333333333333334 None
Con cuda
[train] Epoch 7/20. Iteration 3/31. Loss: 0.50. Accuracy: 0.85					 0.4989978149533272 0.85 None
Con cuda
[train] Epoch 7/20. Iteration 4/31. Loss: 0.53. Accuracy: 0.86					 0.5291109383106232 0.86 None
Con cuda
[train] Epoch 7/20. Iteration 5/31. Loss: 0.63. Accuracy: 0.85					 0.63312000532945 0.85 None
Con cuda
[train] Epoch 7/20. Iteration 6/31. Loss: 0.76. Accuracy: 0.83					 0.7606512350695473 0.8285714285714286 None
Con cuda
[train] Epoch 7/20. Iteration 7/31. Loss: 0.83. Accuracy: 0.79					 0.8325201459228992 0.7875 None
Con cuda
[train] Epoch 7/20. Iteration 8/31. Loss: 0.91. Accuracy: 0.76					 0.9121268490950266 0.7555555555555555 None
Con cuda
[train

[train] Epoch 10/20. Iteration 0/31. Loss: 0.71. Accuracy: 0.80					 0.7091712355613708 0.8 None
Con cuda
[train] Epoch 10/20. Iteration 1/31. Loss: 0.77. Accuracy: 0.85					 0.7695148587226868 0.85 None
Con cuda
[train] Epoch 10/20. Iteration 2/31. Loss: 0.65. Accuracy: 0.87					 0.6503223379453024 0.8666666666666667 None
Con cuda
[train] Epoch 10/20. Iteration 3/31. Loss: 0.73. Accuracy: 0.82					 0.7301848083734512 0.825 None
Con cuda
[train] Epoch 10/20. Iteration 4/31. Loss: 0.69. Accuracy: 0.84					 0.6874608159065246 0.84 None
Con cuda
[train] Epoch 10/20. Iteration 5/31. Loss: 0.74. Accuracy: 0.82					 0.7364744941393534 0.8166666666666667 None
Con cuda
[train] Epoch 10/20. Iteration 6/31. Loss: 0.71. Accuracy: 0.83					 0.7092812742505755 0.8285714285714286 None
Con cuda
[train] Epoch 10/20. Iteration 7/31. Loss: 0.71. Accuracy: 0.81					 0.7089139372110367 0.8125 None
Con cuda
[train] Epoch 10/20. Iteration 8/31. Loss: 0.70. Accuracy: 0.80					 0.7020212544335259 0.8 None
Con

Con cuda
[train] Epoch 11/20. Iteration 24/31. Loss: 0.67. Accuracy: 0.81					 0.6704054689407348 0.812 None
Con cuda
[train] Epoch 11/20. Iteration 25/31. Loss: 0.67. Accuracy: 0.82					 0.6668578661405123 0.8153846153846154 None
Con cuda
[train] Epoch 11/20. Iteration 26/31. Loss: 0.68. Accuracy: 0.81					 0.678277591864268 0.8074074074074075 None
Con cuda
[train] Epoch 11/20. Iteration 27/31. Loss: 0.67. Accuracy: 0.81					 0.6678050543580737 0.8142857142857143 None
Con cuda
[train] Epoch 11/20. Iteration 28/31. Loss: 0.66. Accuracy: 0.81					 0.6629532781140558 0.8137931034482758 None
Con cuda
[train] Epoch 11/20. Iteration 29/31. Loss: 0.67. Accuracy: 0.81					 0.6685621519883473 0.8133333333333334 None
Con cuda
[train] Epoch 11/20. Iteration 30/31. Loss: 0.67. Accuracy: 0.81					 0.6697385994014361 0.8145695364238411 None
Con cuda
[test] Epoch 11/20. Iteration 0/16. Loss: 0.37. Accuracy: 0.90					 0.369832843542099 0.9 None
Con cuda
[test] Epoch 11/20. Iteration 1/16. Loss: 0.37.

[train] Epoch 14/20. Iteration 23/31. Loss: 0.54. Accuracy: 0.85					 0.5411356631666422 0.8458333333333333 None
Con cuda
[train] Epoch 14/20. Iteration 24/31. Loss: 0.54. Accuracy: 0.85					 0.5388423126935958 0.848 None
Con cuda
[train] Epoch 14/20. Iteration 25/31. Loss: 0.53. Accuracy: 0.85					 0.5260428144381597 0.85 None
Con cuda
[train] Epoch 14/20. Iteration 26/31. Loss: 0.53. Accuracy: 0.85					 0.5285132262441847 0.8518518518518519 None
Con cuda
[train] Epoch 14/20. Iteration 27/31. Loss: 0.56. Accuracy: 0.84					 0.556129202246666 0.8428571428571429 None
Con cuda
[train] Epoch 14/20. Iteration 28/31. Loss: 0.55. Accuracy: 0.84					 0.5546779406481775 0.8413793103448276 None
Con cuda
[train] Epoch 14/20. Iteration 29/31. Loss: 0.56. Accuracy: 0.84					 0.5560759286085765 0.8366666666666667 None
Con cuda
[train] Epoch 14/20. Iteration 30/31. Loss: 0.57. Accuracy: 0.83					 0.5663949065650535 0.8311258278145696 None
Con cuda
[test] Epoch 14/20. Iteration 0/16. Loss: 0.36. Accu

Con cuda
[train] Epoch 17/20. Iteration 22/31. Loss: 0.57. Accuracy: 0.87					 0.5697743970414867 0.8739130434782608 None
Con cuda
[train] Epoch 17/20. Iteration 23/31. Loss: 0.56. Accuracy: 0.88					 0.556993575145801 0.8791666666666667 None
Con cuda
[train] Epoch 17/20. Iteration 24/31. Loss: 0.55. Accuracy: 0.88					 0.5516854035854339 0.88 None
Con cuda
[train] Epoch 17/20. Iteration 25/31. Loss: 0.60. Accuracy: 0.88					 0.601869196845935 0.8769230769230769 None
Con cuda
[train] Epoch 17/20. Iteration 26/31. Loss: 0.58. Accuracy: 0.88					 0.584882268751109 0.8814814814814815 None
Con cuda
[train] Epoch 17/20. Iteration 27/31. Loss: 0.59. Accuracy: 0.88					 0.5923075787723064 0.8785714285714286 None
Con cuda
[train] Epoch 17/20. Iteration 28/31. Loss: 0.59. Accuracy: 0.88					 0.5878415328675303 0.8793103448275862 None
Con cuda
[train] Epoch 17/20. Iteration 29/31. Loss: 0.58. Accuracy: 0.88					 0.5847285911440849 0.88 None
Con cuda
[train] Epoch 17/20. Iteration 30/31. Loss: 0.

[train] Epoch 20/20. Iteration 20/31. Loss: 0.42. Accuracy: 0.87					 0.4226731952457201 0.8714285714285714 None
Con cuda
[train] Epoch 20/20. Iteration 21/31. Loss: 0.41. Accuracy: 0.88					 0.4145948639647527 0.8772727272727273 None
Con cuda
[train] Epoch 20/20. Iteration 22/31. Loss: 0.41. Accuracy: 0.87					 0.41470915435448935 0.8739130434782608 None
Con cuda
[train] Epoch 20/20. Iteration 23/31. Loss: 0.43. Accuracy: 0.87					 0.43411764844010275 0.8666666666666667 None
Con cuda
[train] Epoch 20/20. Iteration 24/31. Loss: 0.44. Accuracy: 0.87					 0.43829454511404037 0.868 None
Con cuda
[train] Epoch 20/20. Iteration 25/31. Loss: 0.44. Accuracy: 0.87					 0.44129700700824076 0.8692307692307693 None
Con cuda
[train] Epoch 20/20. Iteration 26/31. Loss: 0.44. Accuracy: 0.87					 0.44250379861504946 0.8666666666666667 None
Con cuda
[train] Epoch 20/20. Iteration 27/31. Loss: 0.46. Accuracy: 0.87					 0.4592445958405733 0.8678571428571429 None
Con cuda
[train] Epoch 20/20. Iteration 2

# Accuracy senza usare il knn

In [33]:
lenet_mnist_predictions, lenet_mnist_gt = test_model_classification(net, barilla_train_loader)
print ("Accuracy aLexNet di train su barillatestloader: %0.2f" % \
accuracy_score(lenet_mnist_gt,lenet_mnist_predictions.argmax(1)))

Accuracy aLexNet di train su barillatestloader: 0.96


In [34]:
lenet_mnist_predictions, lenet_mnist_gt = test_model_classification(net, barilla_test_loader)
print ("Accuracy aLexNet di test su barillatestloader: %0.2f" % \
accuracy_score(lenet_mnist_gt,lenet_mnist_predictions.argmax(1)))

Accuracy aLexNet di test su barillatestloader: 0.57


# Creiamo il dataframe e il knn

In [35]:
knn_1 = KNN(n_neighbors=1)
knn_3 = KNN(n_neighbors=3)
knn_5 = KNN(n_neighbors=5)

if torch.cuda.is_available():
    net = net.cuda()
    torch.cuda.empty_cache()
net.eval()

barilla_train_loader_OB = torch.utils.data.DataLoader(barilla_train, batch_size=1, num_workers=0, shuffle=True)
barilla_test_loader_OB = torch.utils.data.DataLoader(barilla_test, batch_size=1, num_workers=0)


input_for_datafram_train, label_array_train = get_dataframe(barilla_train_loader_OB, net)
df = pd.DataFrame(input_for_datafram_train)
knn_1.fit(df, label_array_train)
knn_3.fit(df, label_array_train)
knn_5.fit(df, label_array_train)

Avviato get_dataframe.
Avviato extract_feature.


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [36]:
feature_test = extract_features(barilla_test_loader_OB, net)
print("Accuracy con rete preallenata e dataset base.")
print("1nn-->",accuracy(knn_1, feature_test))
print("3nn-->",accuracy(knn_3, feature_test))
print("5nn-->",accuracy(knn_5, feature_test))

Avviato extract_feature.
Accuracy con rete preallenata e dataset base.
1nn--> 0.5131578947368421
3nn--> 0.506578947368421
5nn--> 0.40789473684210525


# -----------------------------------------------------------------------------------------------

# Rialleniamo alexnet con 153 classi in output. 20 epoche.


In [44]:
net = models.alexnet()
net.classifier[6] = nn.Linear(4096, 153) #Numero esatto di classi nel nostro dataset.
sum([p.numel() for p in net.parameters()])

57630681

In [45]:
lenet_mnist, lenet_mnist_logs = train_classification(net, epochs=3, train_loader = barilla_train_loader,
                                                     test_loader = barilla_test_loader)

Con cuda
[train] Epoch 1/3. Iteration 0/31. Loss: 5.04. Accuracy: 0.00					 5.038596153259277 0.0 None
Con cuda
[train] Epoch 1/3. Iteration 1/31. Loss: 5.03. Accuracy: 0.00					 5.032897472381592 0.0 None
Con cuda
[train] Epoch 1/3. Iteration 2/31. Loss: 5.03. Accuracy: 0.00					 5.0323710441589355 0.0 None
Con cuda
[train] Epoch 1/3. Iteration 3/31. Loss: 5.03. Accuracy: 0.00					 5.0317264795303345 0.0 None
Con cuda
[train] Epoch 1/3. Iteration 4/31. Loss: 5.03. Accuracy: 0.00					 5.031992435455322 0.0 None
Con cuda
[train] Epoch 1/3. Iteration 5/31. Loss: 5.03. Accuracy: 0.00					 5.031418800354004 0.0 None
Con cuda
[train] Epoch 1/3. Iteration 6/31. Loss: 5.03. Accuracy: 0.00					 5.031723090580532 0.0 None
Con cuda
[train] Epoch 1/3. Iteration 7/31. Loss: 5.03. Accuracy: 0.00					 5.031984031200409 0.0 None
Con cuda
[train] Epoch 1/3. Iteration 8/31. Loss: 5.03. Accuracy: 0.00					 5.032042238447401 0.0 None
Con cuda
[train] Epoch 1/3. Iteration 9/31. Loss: 5.03. Accuracy: 0.00	

[train] Epoch 2/3. Iteration 27/31. Loss: 5.03. Accuracy: 0.01					 5.030723367418561 0.007142857142857143 None
Con cuda
[train] Epoch 2/3. Iteration 28/31. Loss: 5.03. Accuracy: 0.01					 5.0307652374793745 0.006896551724137931 None
Con cuda
[train] Epoch 2/3. Iteration 29/31. Loss: 5.03. Accuracy: 0.01					 5.030816141764323 0.006666666666666667 None
Con cuda
[train] Epoch 2/3. Iteration 30/31. Loss: 5.03. Accuracy: 0.01					 5.030832821170226 0.006622516556291391 None
Con cuda
[test] Epoch 2/3. Iteration 0/16. Loss: 5.03. Accuracy: 0.00					 5.031169414520264 0.0 None
Con cuda
[test] Epoch 2/3. Iteration 1/16. Loss: 5.03. Accuracy: 0.00					 5.031979322433472 0.0 None
Con cuda
[test] Epoch 2/3. Iteration 2/16. Loss: 5.03. Accuracy: 0.00					 5.030284722646077 0.0 None
Con cuda
[test] Epoch 2/3. Iteration 3/16. Loss: 5.03. Accuracy: 0.00					 5.031891465187073 0.0 None
Con cuda
[test] Epoch 2/3. Iteration 4/16. Loss: 5.03. Accuracy: 0.00					 5.031502056121826 0.0 None
Con cuda
[test

# Accuracy senza usare il knn

In [46]:
lenet_mnist_predictions, lenet_mnist_gt = test_model_classification(net, barilla_train_loader)
print ("Accuracy aLexNet di train su barillatestloader: %0.2f" % \
accuracy_score(lenet_mnist_gt,lenet_mnist_predictions.argmax(1)))

Accuracy aLexNet di train su barillatestloader: 0.01


In [47]:
lenet_mnist_predictions, lenet_mnist_gt = test_model_classification(net, barilla_test_loader)
print ("Accuracy aLexNet di test su barillatestloader: %0.2f" % \
accuracy_score(lenet_mnist_gt,lenet_mnist_predictions.argmax(1)))

Accuracy aLexNet di test su barillatestloader: 0.01


# Creiamo il dataframe e il knn

In [ ]:
knn_1 = KNN(n_neighbors=1)
knn_3 = KNN(n_neighbors=3)
knn_5 = KNN(n_neighbors=5)

if torch.cuda.is_available():
    net = net.cuda()
    torch.cuda.empty_cache()
net.eval()

barilla_train_loader_OB = torch.utils.data.DataLoader(barilla_train, batch_size=1, num_workers=0, shuffle=True)
barilla_test_loader_OB = torch.utils.data.DataLoader(barilla_test, batch_size=1, num_workers=0)


input_for_datafram_train, label_array_train = get_dataframe(barilla_train_loader_OB, net)
df = pd.DataFrame(input_for_datafram_train)
knn_1.fit(df, label_array_train)
knn_3.fit(df, label_array_train)
knn_5.fit(df, label_array_train)

Avviato get_dataframe.
Avviato extract_feature.


In [ ]:
feature_test = extract_features(barilla_test_loader_OB, net)
print("Accuracy con rete preallenata e dataset base.")
print("1nn-->",accuracy(knn_1, feature_test))
print("3nn-->",accuracy(knn_3, feature_test))
print("5nn-->",accuracy(knn_5, feature_test))

# -----------------------------------------------------------------------------------------------

# Alexnet preallenata con data augmentation e 1nn
<br>
# Prendiamo media e varianza per normalizzare le immagini

In [ ]:
#dataset = ScenesDataset('Dataset_data_augmentation','train_data_augmentation.txt',transform=transforms.ToTensor())
#m, s = get_mean_devst(dataset)

# Creiamo il dataframe e il knn

In [ ]:
knn_1_da = KNN(n_neighbors=1)

#if torch.cuda.is_available():
     #net = net.cuda()
    #torch.cuda.empty_cache()
net.cpu()
net.eval()

input_for_datafram_train_da, label_array_train_da = get_dataframe(barilla_train_loader_da, net)
df_da = pd.DataFrame(input_for_datafram_train_da)
knn_1_da.fit(df_da, label_array_train_da)

# Prendiamo l'accuracy di test

In [ ]:
feature_test_da = extract_features(barilla_test_loader_da, net)
print("Accuracy con rete preallenata e dataset base.")
print(accuracy(knn_1_da, feature_test_da))
#Con 1nn accuracy di 0.36348684210526316
#Con 3nn accuracy di 0.3873355263157895
#Con 5nn accuracy di 0.37335526315789475

# Utilizziamo AlexNet con i parametri pre-allenati, eliminiamo il layer di classificazione e ci facciamo dare in output un array di 64 elementi


In [ ]:

net = models.alexnet(pretrained=True)
#classifier = nn.Sequential(*list(net.classifier.children())[:-1])
classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 153), #Numero esatto di classi nel nostro dataset.
        )
net.classifier = classifier
sum([p.numel() for p in net.parameters()])

In [ ]:
knn_1 = KNN(n_neighbors=1)

if torch.cuda.is_available():
    net = net.cuda()
    torch.cuda.empty_cache()
net.eval()

input_for_datafram_train, label_array_train = get_dataframe(barilla_train_loader, net)
df = pd.DataFrame(input_for_datafram_train)
knn_1.fit(df, label_array_train)

In [ ]:
feature_test = extract_features(barilla_test_loader, net)
print("Accuracy con rete preallenata e dataset base.")
print(accuracy(knn_1, feature_test))
#Accuracy con ultimo layer che in output ci da 64: 0.1907
#Accuracy con ultimo layer che in output ci da 128: 0.1970
#Accuracy con ultimo layer che in output ci da 256: 0.2039


# Prova riallenamento.

# Funzione per riallenare.

In [ ]:
net = models.alexnet(pretrained=True)
#classifier = nn.Sequential(*list(net.classifier.children())[:-1])

for param in net.parameters():
    param.requires_grad = False
net.classifier[6] = nn.Linear(4096, 153) #Numero esatto di classi nel nostro dataset.
sum([p.numel() for p in net.parameters()])

In [ ]:
lenet_mnist, lenet_mnist_logs = train_classification(net, epochs=3, train_loader = barilla_train_loader_da,
                                                     test_loader = barilla_test_loader_da)

# Per prendere accuracy dalla rete senza usare il knn

# Accuracy senza usare il knn

In [ ]:
lenet_mnist_predictions, lenet_mnist_gt = test_model_classification(net, barilla_train_loader_da)
print ("Accuracy aLexNet di train su barillatestloader: %0.2f" % \
accuracy_score(lenet_mnist_gt,lenet_mnist_predictions.argmax(1)))
#Accuracy di train con dataset data agumentation, ultimo layer riallenato  e 40 epoche di 0.80
#Accuracy di train con dataset data agumentation, ultimi due layer riallenati  e 2 epoche di 0.07
#Accuracy di train con dataset data agumentation, dropout, ultimo layer allenato e 20 epoche di 0.56

In [ ]:
lenet_mnist_predictions, lenet_mnist_gt = test_model_classification(net, barilla_test_loader_da)
print ("Accuracy aLexNet di test su barillatestloader: %0.2f" % \
accuracy_score(lenet_mnist_gt,lenet_mnist_predictions.argmax(1)))
#Accuracy di test con dataset data agumentation, ultimo layer riallenato  e 40 epoche di 0.47
#Accuracy di test con dataset data agumentation, ultimi due layer riallenati  e 2 epoche di 0.07
#Accuracy di test con dataset data agumentation, dropout, ultimo layer allenato e 20 epoche di 0.3'

# Accuracy da Knn con rete che ha in output 153 classi

# AL MOMENTO NON FUNZIONANTE.

In [ ]:
knn_1 = KNN(n_neighbors=5)

if torch.cuda.is_available():
    net = net.cuda()
    torch.cuda.empty_cache()
net.eval()
#
barilla_train_loader_OB = torch.utils.data.DataLoader(barilla_train_da, batch_size=1, num_workers=0, shuffle=True)
barilla_test_loader_OB = torch.utils.data.DataLoader(barilla_test_da, batch_size=1, num_workers=0)

input_for_datafram_train, label_array_train = get_dataframe(barilla_train_loader_OB, net)
df = pd.DataFrame(input_for_datafram_train)
knn_1.fit(df, label_array_train)
feature_test = extract_features(barilla_test_loader_OB, net)
print("Accuracy con rete preallenata e dataset base.")
print(accuracy(knn_1, feature_test))
#Accuracy di test con dataset data agumentation, ultimi due layer riallenati  e 2 epoche di 0.19

#Accuracy di test con dataset data agumentation, dropout, ultimo layer allenato 20 epoche e 1 vicini di 0.31
#Accuracy di test con dataset data agumentation, dropout, ultimo layer allenato 20 epoche e 3 vicini di 0.377
#Accuracy di test con dataset data agumentation, dropout, ultimo layer allenato 20 epoche e 5 vicini di 0.40

In [ ]:
torch.save(net.state_dict(), "./50epoche153classi.pth")